# Preprocessamento de Dados e Armazenamento no S3

Este notebook demonstra como carregar, pré-processar e dividir um dataset de clientes para modelagem de churn. Utilizamos o **Amazon S3** para armazenar os arquivos de treino e teste.

As etapas incluem:
1. Carregar o dataset diretamente do S3.
2. Realizar o pré-processamento dos dados (exclusão de colunas, tratamento de valores nulos, conversão de variáveis categóricas).
3. Separar os dados em conjuntos de treino e teste.
4. Salvar os conjuntos de treino e teste de volta no S3.

---
## 1 - Bibliotecas Necessárias

Antes de começar, importamos as bibliotecas necessárias, como o **boto3** para acessar o Amazon S3 e o **pandas** para manipulação dos dados.


In [2]:
import boto3
import pandas as pd
from io import StringIO
from sklearn.model_selection import train_test_split
# Conectar ao S3
s3 = boto3.client('s3')


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.sparse import csr_matrix, issparse


In [8]:
bucket_name = 'sagemaker-us-east-1-837612112902'
input_key = 'telco_churn/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv'
train_output_key = 'telco_churn/dataset/train.csv'
test_output_key = 'telco_churn/dataset/test.csv'

## 2 - Conexão ao S3 e Carregamento do Dataset

In [ ]:
# Ler o dataset do S3
#df = read_s3_csv(bucket_name, input_key)

df = pd.read_csv('s3://{}/{}'.format(bucket_name, input_key))

# Excluir a coluna 'customerID' (se existir)
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])

# Converter a variável de resposta 'Churn' para binário
if 'Churn' in df.columns:
    df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Converter a coluna 'TotalCharges' para numérico e tratar valores faltantes
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)



## 3 - Separação de Treino e Teste

In [ ]:
# Separar as features (X) da variável alvo (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Criar DataFrames para treino e teste
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)





## 4 - Salvando os Arquivos no S3

In [ ]:
# Salvar os arquivos de treino e teste no S3
train_data.to_csv('s3://{}/{}'.format(bucket_name, train_output_key), index=False)
test_data.to_csv('s3://{}/{}'.format(bucket_name, test_output_key), index=False)

print(f"Arquivos de treino e teste salvos com sucesso no S3 em:\n - {train_output_key}\n - {test_output_key}")

## 5 - Carregar Arquivos do S3 (Teste)

In [ ]:
train_data = pd.read_csv('s3://{}/{}'.format(bucket_name, train_output_key))
test_data = pd.read_csv('s3://{}/{}'.format(bucket_name, test_output_key))